# Script presenting data for checking

In [2]:
import plotly.graph_objs as go
import pandas as pd
import os

In [ ]:
Solcast_Path = '../Renewables2023_Private/Private_Data/Solcast/Solcast_Formatted.csv'
Public = False

# Checking if the necessary files exist for formatting.
if not os.path.exists(Solcast_Path):
    print(f"Warning: Some of this script will not run without the formatted Solcast data in {Solcast_Path}`.\n",
          'Please download the data from `solcast.com` at 5 minute cadence for all of 2023.\n',
          'Then run the script in `Scripts/Utils/Format_Solcast.ipynb` to format the data.')
    Public = True

In [ ]:
DEOP = pd.read_csv('../Data/DEOP/2023_DEOP_Interp.csv',
                    parse_dates=['DateTime'],
                    index_col=['DateTime']).astype(float)

if Public is False:
    Solcast = pd.read_csv(Solcast_Path,
                        parse_dates=['DateTime'],
                        index_col=['DateTime']).astype(float)
    
    Power_Expectation_Path = '../Renewables2023_Private/Private_Data/Solcast/Power_Expectation.csv'

    if not os.path.exists(Power_Expectation_Path):
        print(f"Warning: This script will not run without the processed data in {Power_Expectation_Path}`.\n",
          'To create this file, please run the script at `Scripts/Renewable_Energy_Efficiency.ipynb`.')
    else:
        PowerExpectation = pd.read_csv(Power_Expectation_Path,
                            parse_dates=['DateTime'],
                            index_col=['DateTime']).astype(float)
else:
    Solcast = None

In [5]:
# Calculating total power generation.
DEOP['power-gencomb-ave'] = (DEOP['power-gen-wt-ave'] + DEOP['power-gen-pv-ave']).clip(lower=0)

# Calculating power excess.
DEOP['power-excess-ave'] = (DEOP['power-gencomb-ave'] - DEOP['power-con-ave']).clip(lower=0)

# Calculating power generation subtracted from power consumption.
DEOP['power-congen-ave'] = (DEOP['power-con-ave'] - DEOP['power-gencomb-ave'])

In [6]:
ColourPallet = ['#000000','#E69F00','#56B4E9','#009E73','#0072B2','#D55E00','#CC79A7']

In [ ]:
if Public is False:  
    Solcast['P_KEWind'] = PowerExpectation['P_KEWind']
    Solcast['P_MaxSun'] = PowerExpectation['P_MaxSolar']

stats = pd.read_csv('../Data/Analysis/Data_Tables/csv/statistics.csv',index_col=0)

In [8]:
def CheckData(dfs,colours,y_label,data_labels,title):
    """
    Function for plotting data to check through it

    Parameters
    ----------
    dfs : List
        List of DataFrames to plot (pd.DataFrame).
    colours : List
        List of colours of lines to plot.
    y_label : str
        y-axis label.
    data_labels : List
        List of labels to describe data in legend (str).
    title : str
        Title of plot.
    """
    fig = go.Figure()
    for i in range(len(dfs)):
        fig.add_trace(go.Scatter(x=dfs[i].index,y=dfs[i],mode='lines',name=data_labels[i],opacity=0.6,line_color=colours[i]))
    
    fig.update_layout(
    title=title,
    xaxis_title='Time',
    yaxis_title=y_label,
    template='plotly_white')
    
    fig.show()
    

In [ ]:
# Example use of the function, plotting true and expected solar power generation.

CheckData([DEOP['power-gen-pv-ave'],Solcast['P_MaxSun']*stats.loc['eta_Solar','mean']],ColourPallet,y_label='Power [kW]',
          data_labels=['Generation','Expectation'],title='True and expected solar power generation.')